> `open_wave(wave_index)` : opens all files from a given wave and returns a list of all the datasets 

> `compute_stats(dataframe, wave)` : calculates mean, median and standard deviation for all quantitative variables for a given wave

In [2]:
import pandas as pd
import os
from functools import reduce
import numpy as np

def open_wave(wave):
    
    PATH = "/Users/josephbarbier/Desktop/PROJETpython/rawdata/wave"+str(wave)
    dfList = []
    
    #iterate over all files that end with ".dta" and add them to a dfList
    for index, dirs, files in os.walk(PATH):
        for file in files:
            if file.endswith(".dta"):
                df = pd.read_stata(
                    os.path.join(index, file),
                    chunksize=1,
                    convert_categoricals=False).read()
                
                #add the current df to the list
                dfList.append(df)
            
    return dfList

In [8]:
#merge and save all data into 1 df per wave
for i in range(1,9):
    
    #merge all dfs into one df
    data = reduce(lambda left, right:
                  pd.merge(left,
                           right,
                           on = ["mergeid"],
                           how = "outer",
                           suffixes = (None, "_y2")),
                    open_wave(i))
    
    #all wave 3 variables start with "sl_" (for SHARE LIFE) and we remove them
    #because otherwise our regex functions don't work
    if i==3:
        data.columns = data.columns.str.replace(r'^sl_', '', regex=True)

    #remove columns containing the "_y2" pattern since they are dupplicate
    data = data[data.columns.drop(list(data.filter(regex='_y2$')))]
    print(f"Shape of wave {i}:", data.shape)
    data.to_csv(f"wave/wave{i}", index=False)
    print(f"The file of wave {i} has been saved!\n")

Shape of wave 1: (30419, 1649)
The file of wave 1 has been saved!

Shape of wave 2: (37143, 1897)
The file of wave 2 has been saved!



/var/folders/m2/jfzxyg4s0xz3jyw5f94r7tcm0000gn/T/ipykernel_2464/1865696712.py:16: FutureWarning: The default value of regex will change from True to False in a future version.
  data.columns = data.columns.str.replace(r'^sl_', '')


Shape of wave 3: (28463, 3008)
The file of wave 3 has been saved!

Shape of wave 4: (58000, 2475)
The file of wave 4 has been saved!

Shape of wave 5: (66065, 3059)
The file of wave 5 has been saved!

Shape of wave 6: (68085, 4193)
The file of wave 6 has been saved!

Shape of wave 7: (77202, 6562)
The file of wave 7 has been saved!

Shape of wave 8: (46733, 4298)
The file of wave 8 has been saved!



# Function that computes mean, sd and median for all quantitative variables

In [169]:
def compute_stats(dataframe, wave):
    
    wave = str(wave)
    
    df = dataframe.copy()
    variable = list(df.columns)

    #init lists to fill in
    mean = []
    sd = []
    median = []
    var_name = []
    
    for var in variable:
        
        #compute mean, sd and median for all quant variables
        df[var] = pd.to_numeric(df[var], errors='coerce')
        mean.append(df[var].mean())
        sd.append(df[var].std())
        median.append(df[var].median())
        var_name.append(var+"_"+wave)
        
    #concatenate all lists and name columns
    features = {'Var_name': var_name,
                'Mean': mean, 
                'Std': sd, 
                'Median': median} 
    df_to_return = pd.DataFrame(features, columns= ['Var_name','Mean','Std','Median'])
    df_to_return = df_to_return.dropna(axis=0)
    
    return df_to_return

In [170]:
#compute and save all results
for wave in tqdm(range(1,9)):
    
    #open data
    data = pd.read_csv(f"wave{wave}.csv", low_memory=False)
    print(data.shape)

    #apply compute_stats function
    test = data.copy()
    stats = compute_stats(test, wave=wave)
    
    #save results
    stats.to_csv(f"stats_wave{wave}.csv", index=False)
    stats.head()

  0%|          | 0/8 [00:00<?, ?it/s]

(30419, 1600)


 12%|█▎        | 1/8 [02:47<19:32, 167.45s/it]

(71305, 2025)


 25%|██▌       | 2/8 [18:57<1:03:56, 639.35s/it]

...

 75%|███████▌  | 6/8 [3:41:48<1:53:45, 3412.93s/it]

(88957, 6526)


 88%|████████▊ | 7/8 [7:00:44<1:43:19, 6199.36s/it]

(58527, 4383)


100%|██████████| 8/8 [9:03:34<00:00, 4076.84s/it]  


In [193]:
all_stats = pd.Series(dtype=np.float64)
#check shape of all stats
for wave in tqdm(range(1,9)):
    data = pd.read_csv(f"stats_wave{wave}.csv", low_memory=False)
    #print(f"Descriptive stats of wave{wave} \n", data.head(), "\n")
    all_stats = pd.concat([all_stats, data])

print(all_stats.head())

100%|██████████| 8/8 [00:00<00:00, 295.36it/s]

    0   Var_name           Mean           Std   Median
0 NaN   exrate_1       2.563018  2.864668e+00      1.0
1 NaN   as003e_1  151841.692249  1.223841e+07   5000.0
2 NaN  as003v1_1    3320.559777  8.666762e+02   3600.0
3 NaN  as003v2_1    6590.547301  1.734869e+03   7100.0
4 NaN  as003v3_1   13208.835095  3.475529e+03  14000.0
